In [1]:
# Fundamentos de Ciência de Dados
## PPGI/UFRJ 2024.2
### Profs Sergio Serra e Jorge Zavaleta
### Aluno Ubirajara S. Santos

In [2]:
import pandas as pd
import prov
import matplotlib

In [ ]:
# Fontes de Dados
data_sources = {
    "amostras_rochas_fluidos": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-amostras-de-rochas-e-fluidos/acervo-de-amostras/consolidacao-2023.zip",
        "type": "zip" 
    },
    "setores_sirgas": {
        "url": "https://www.gov.br/anp/pt-br/assuntos/exploracao-e-producao-de-oleo-e-gas/estudos-geologicos-e-geofisicos/arquivos-classificacao-de-modelos-exploratorios/setores-sirgas.zip",
        "type": "zip"
    },
    "blocos_exploratorios": {
        "url": "https://gishub.anp.gov.br/geoserver/BD_ANP/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=BD_ANP%3ABLOCOS_EXPLORATORIOS_SIRGAS&maxFeatures=40000&outputFormat=SHAPE-ZIP",
        "type": "zip" 
    },
    "campos_producao": {
        "url": "https://gishub.anp.gov.br/geoserver/BD_ANP/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=BD_ANP%3ACAMPOS_PRODUCAO_SIRGAS&maxFeatures=40000&outputFormat=SHAPE-ZIP",
        "type": "zip"
    },
    "reservas_nacionais": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-estatisticos/arquivos-reservas-nacionais-de-petroleo-e-gas-natural/boletim-anual-reservas-2023.pdf",
        "type": "pdf"
    },
    "reservas_nacionais_hc": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-estatisticos/arquivos-reservas-nacionais-de-petroleo-e-gas-natural/tabela-dados-bar-2023.xlsx",
        "type": "xlsx" 
    },
    "tabela_pocos_2024": {
        "url": "./dados/entradas/Tabela_pocos_2024_Novembro_24.csv",
        "type": "csv",
        "sep": ";"  # Separador do arquivo CSV
    }
}